#### Converting Llama 2 to Llama 3.2 From Scratch

In [ ]:
# pip install -r requirements-extra.txt

###### Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

pkgs = [
    "blobfile",     # to download pretrained weights
    "huggingface_hub",  # to download pretrained weights
    "tiktoken",         # to implement the tokenizer
    "torch",            # to implement the model
]
for p in pkgs:
    print(f"{p} version: {version(p)}")

#### 1. Convert the Llama model implementation step by step

###### The Converting a From-Scratch GPT Architecture to Llama 2 then implements the Llama-specific components, such as RMSNorm layers, SiLU and SwiGLU activations, RoPE (rotary position embeddings), and the SentencePiece tokenizer
###### This notebook takes the Llama 2 architecture and transforms it into Llama 3 architecture by
###### - modifying the rotary embeddings
###### - implementing grouped-query attention
###### - and using a customized version of the GPT-4 tokenizer
###### Later, we then load the original Llama 3 weights shared by Meta AI into the architecture

##### 1.1 Reusing Llama 2 components
###### Llama 2 is actually quite similar to Llama 3, as mentioned above and illustrated in the figure at the top of this notebook
###### This means that we can import several building blocks from the Llama 2 notebook using the following code

In [ ]:
import os
import sys
import io
import nbformat
import types

def import_from_notebook():
    def import_definitions_from_notebook(fullname, names):
        current_dir = os.getcwd()
        path = os.path.join(current_dir, fullname + ".ipynb")
        path = os.path.normpath(path)

        # Load the notebook
        if not os.path.exists(path):
            raise FileNotFoundError(f"Notebook file not found at: {path}")

        with io.open(path, "r", encoding="utf-8") as f:
           nb = nbformat.read(f, as_version=4)

        # Create a module to store the imported functions and classes
        mod = types.ModuleType(fullname)
        sys.modules[fullname] = mod

        # Go through the notebook cells and only execute function or class definitions
        for cell in nb.cells:
            if cell.cell_type == "code":
                cell_code = cell.source
                for name in names:
                    # Check for function or class definitions
                    if f" def {name}" in cell_code or f"class {name}" in cell_code:
                        exec(cell_code, mod.__dict__)
        return mod

    fullname = "converting-gpt-to-llama2"
    names = ["precompute_rope_params", "compute_rope", "SiLU", "FeedForward", "RMSNorm"]